In [ ]:
DROP TABLE IF EXISTS amzl_mapping;
CREATE TEMP TABLE amzl_mapping  AS (
    SELECT
        location_id AS node,
        timezone,
        region,
        country,
        GETDATE() AS run_time_utc,
        CAST(CAST(GETDATE() AS DATE) AS TIMESTAMP) AS day_start_utc,
        CONVERT_TIMEZONE('UTC', timezone, GETDATE()) AS run_time_local,
        CONVERT_TIMEZONE('UTC', timezone, CAST(CAST(GETDATE() AS DATE) AS TIMESTAMP)) AS local_offset_utc
    FROM
        "amzlanalytics"."perfectmile"."d_perfectmile_node_mapping_mdm"
    WHERE
        1=1
        AND location_status = 'A'
        AND country IN('US', 'CA')
        -- Uncomment to filter by specific regions if needed
        -- AND UPPER(region) IN('ROCKIES', 'UPSTATE NY') 
        AND (location_type = 'DS' 
            OR location_id IN ('MCO5', 'ZYG1', 'ZYN9', 'XVV2', 'XVV3', 'XYT6', 'XLC1', 'XVC1', 'XNK2')
        )
);

In [ ]:
-- Step 2: Create a temp table for slam legs

DROP TABLE IF EXISTS slam_leg;
CREATE TEMP TABLE slam_leg SORTKEY(shipment_key) DISTKEY (shipment_key) AS (
    SELECT a.*,
           'slam_leg' AS source_table,
           m.timezone,
           m.region,
           m.country,
           m.run_time_utc,
           m.day_start_utc,
           m.run_time_local,
           m.local_offset_utc
    FROM (
        SELECT
            shipment_id,
            package_id,
            shipment_id ||'#'|| package_id as shipment_key,
            route_id,
            route_warehouse_id,
            route_ship_method,
            route_internal_sort_code,
            route_external_sort_code,
            processing_date,
            leg_sequence_id,
            leg_id,
            leg_warehouse_id,
            leg_ship_method,
            leg_internal_sort_code,
            leg_external_sort_code,
            leg_destination_warehouse_id,
            ship_option,
            pickup_date,
            estimated_arrival_date,
            transit_time_in_hours,
            "zone",
            request_timestamp,
            dw_creation_date,
            GETDATE() AS unloaded_at,
            ROW_NUMBER() OVER (PARTITION BY shipment_id ORDER BY request_timestamp DESC, pickup_date DESC) AS leg_rn
        FROM backlog_datasets.ATROPS.o_slam_packages_leg_live
        WHERE request_timestamp > DATE_TRUNC('DAY', CONVERT_TIMEZONE('UTC', 'America/Chicago', GETDATE())) - INTERVAL '2 DAYS'
    ) a
    LEFT JOIN amzl_mapping m
    ON m.node = a.leg_warehouse_id
    WHERE a.leg_rn = 1
);

In [ ]:

DROP TABLE IF EXISTS gmp;
CREATE TEMP TABLE gmp INTERLEAVED SORTKEY (shipment_key, tracking_id) DISTKEY (tracking_id) AS (
SELECT
    shipment_type,
    sender_id,
    tracking_id,
    ship_track_event_code,
    standard_carrier_alpha_code,
    supplement_code,
    tcda_container_id,
    parent_container_id,
    parent_container_type,
    status_node_id,
    load_id,
    status_date,
    status_date_timezone,
    edi_standard_name,
    status_code,
    reason_code,
    fulfillment_reference_id,
    marketplace_id,
    amazon_bar_code,
    fulfillment_shipment_id,
    package_id,
    estimated_arrival_date,
    promised_arrival_date,
    tss_ship_date,
    business_unit,
    ship_option,
    ship_method,
    actual_delivery_date,
    attempted_delivery_date,
    is_export_charge_prepaid,
    is_virtual_scan,
    access_point_id,
    seller_id,
    transport_shipment_id,
    amazon_reference_number,
    additional_reference_number,
    status_ref_target_type,
    predicted_delivery_date,
    estimated_delivery_date,
    origin_address_country_code
    manifest_id,
    service_type,
    dw_created_time,
    GETDATE() AS unloaded_at,
    fulfillment_shipment_id || '#' || package_id AS shipment_key,
    shipment_key || '#' || tracking_id as shipment_tracking_key,
    ROW_NUMBER() OVER(PARTITION BY tracking_id ORDER BY status_date DESC) as status_date_rn,
    -- ROW_NUMBER() OVER(PARTITION BY tracking_id, status_date ORDER BY )
    1 as duplicate_rn,
    'GMP' as source_table
FROM
    "backlog_datasets"."amzlcore"."gmp_shipment_events_na" GMP
WHERE
    1=1
    AND GMP.dw_created_time > DATE_TRUNC('DAY', CONVERT_TIMEZONE('UTC', 'America/Chicago', GETDATE())) - INTERVAL '2 DAYS'
    AND (
        GMP.ship_track_event_code IN (
        'EVENT_108',    -- SWA Pickup Failure
        'EVENT_250',    -- Sender cancelled pick up (MFN?)
        'EVENT_253',    -- ATS Arrival
        'EVENT_254',    -- ATS Departure
        'EVENT_258',    -- Identify shipments that can be delivered beyond FRD and publish a new EDD to Customers
        'EVENT_301',    -- Delivery
        'EVENT_302',    -- Dispatch
        'EVENT_304',    -- Attempt
        'EVENT_407',    -- Recipient refused (MFN?)
        'EVENT_408',    -- Shipment is undeliverable and is returning to Amazon or to MFN seller
        'EVENT_476',    -- Potential lost
        'EVENT_760',    -- Shipment damaged and will be replaced
        'EVENT_762',    -- Missorted and will be returned
        'EVENT_764'     -- Mislabled and will be returned
        -- 'EVENT_201'                -- updating this for middle mile visibility. Woot (dropship?) only available at slam. Delivery depot check is redundant, makes code more difficult
        -- 'EVENT_202',    -- Departure -- removing due to no final_destination or vrid join available
        -- 'EVENT_208',    -- SWA, need to classify (5/25/25)
        -- 'EVENT_503'     -- Secured pick up appointment date || AGL amazon global listing, spot check occured at slam || https://w.amazon.com/bin/view/TxNotifications/upstream/AGL
        -- EVENT_102, 'EVENT_103', 'EVENT_104', 'EVENT_216', 'EVENT_228', 'EVENT_238', 'EVENT_307', 'EVENT_308', 'EVENT_310', 'EVENT_320', 'EVENT_402', 'EVENT_404', 'EVENT_660', 'EVENT_661'
        )
        OR
        (GMP.ship_track_event_code = 'EVENT_201' AND GMP.supplement_code = 'AMAZON_FACILITY') -- updating this for middle mile visibility. Woot, presumably dropship will show the shipment's arrival at fc (presumably confirming it was received from seller. So a confirmation after slam)
    )
);




In [ ]:
DROP TABLE IF EXISTS pse;
CREATE TEMP TABLE pse INTERLEAVED SORTKEY (forward_tracking_id, state_status) DISTKEY (forward_tracking_id) AS (
SELECT
      PSE."type"
    , PSE.package_id
    , PSE.package_id_type
    , PSE.forward_amazon_barcode
    , PSE.forward_tracking_id
    , PSE.forward_tcda_container_id
    , PSE.state_location_type
    , PSE.state_location_id
    , PSE.state_location_destination_id
    , PSE.state_location_source_id
    , PSE.state_status
    , PSE.state_time
    , PSE.triggerer_id
    , PSE.triggerer_id_type
    , PSE.dw_created_time
    , PSE.state_sub_status
    , PSE.comp_type
    , PSE.comp_reason
    , PSE.comp_state
    , PSE.reverse_amazon_barcode
    , PSE.reverse_tcda_container_id
    , PSE.reverse_tracking_id
    , PSE.execution_id
    , PSE.execution_id_type
    , PSE.state_location_destination_type
    , PSE.state_location_source_type
    , ROW_NUMBER() OVER(PARTITION BY PSE.forward_tracking_id ORDER BY PSE.state_time DESC) AS state_time_rn
    , ROW_NUMBER() OVER(PARTITION BY forward_tcda_container_id, forward_tracking_id ORDER BY state_time DESC, reverse_tracking_id) AS forward_tcda_rn
    , 'PSE' as source_table
    , GETDATE() AS unloaded_at
    , 1 as duplicate_rn
FROM
    "backlog_datasets"."amzlcore"."package_systems_event_na" PSE
                                                                                                                                                                                            -- INNER JOIN
                                                                                                                                                                                                -- (SELECT DISTINCT shipment_package_tracking_pk, tracking_id from gmp where tracking_id IS NOT NULL) GMP -- this join assumes that a tracking_id cannot be associated with >1 shipment_id
                                                                                                                                                                                            -- ON
                                                                                                                                                                                                -- PSE.forward_tracking_id = GMP.tracking_id

WHERE
    1=1
    AND dw_created_time > DATE_TRUNC('DAY', CONVERT_TIMEZONE('UTC', 'America/Chicago', GETDATE())) - INTERVAL '2 DAYS'
);

In [ ]:
DROP TABLE IF EXISTS node_type;
CREATE TABLE node_type SORTKEY (location_id) DISTKEY (location_id) AS (
SELECT
DISTINCT
      location_id
    , location_type
FROM
    "amzlanalytics"."perfectmile"."d_perfectmile_node_mapping_mdm"
WHERE 1=1
    AND location_id SIMILAR TO '[A-Z]{3}[A-Z0-9]'
ORDER BY
      location_type
    , location_id
);

In [ ]:
DROP TABLE IF EXISTS public.nc_load_summary_snapshot;
CREATE TABLE public.nc_load_summary_snapshot DISTKEY(vrid) sortkey(final_destination) as (

SELECT
      vrid
    , origin
    , final_destination
    , origin_type
    , destination_type
    , lane
    , cpt
    , miles
    , origin_local_timezone
    , origin_scheduled_arrival
    , origin_calc_arrival
    , origin_arrival_late_group
    , origin_arrival_late_hrs
    , origin_arrival_reason
    , origin_scheduled_depart
    , origin_calc_depart
    , origin_departure_late_group
    , origin_depart_late_hrs
    , dest_local_timezone
    , dest_scheduled_arrival
    , dest_calc_arrival
    , dest_arrival_late_group
    , dest_arrival_late_hrs
    , dest_arrival_reason
    , account_id
    , report_day
    , vr_create_date
    -- , ROW_NUMBER() OVER(PARTITION BY vrid ORDER BY vr_create_date DESC) as duplicate_rn
    , 'VLS' as sub_source
FROM
    "andes"."ats-onestopshop"."v_load_summary_hourly"
WHERE
    1=1
    AND region_id = 'NA'
    AND dest_country IN ('US', 'CA')
    AND report_day > GETDATE() - INTERVAL '14 DAYS'
-- QUALIFY
--     ROW_NUMBER() OVER(PARTITION BY vrid ORDER BY vr_create_date DESC) = 1 -- this should not be neccesary

);